# Metadata

```yaml
Course:   DS5001: Exploratory Text Analytics
Topic:    Final Project, Data Prep
Author:   Andrew Avitabile
Date:     24 March 2024 (Edited April 25, 2024)
```

# Set Up

## Packages

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from collections import Counter

#nltk packages
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#Sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Downloading necessary data from nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('vader_lexicon')


# Creating a list of stop words for later use
stop_words = set(stopwords.words('english'))

# Initialize Porter Stemmer
stemmer = PorterStemmer()

In [2]:
# Define the base path
base_path = "C:/Users/Andre/Box/DS5001 Final Project/"

## Import Data

In [3]:
# Define filepaths
file_path_eval_text = base_path + "Data/eval_text.xlsx"

# Read the CSV file
eval_text = pd.read_excel(file_path_eval_text)

In [4]:
#Get just PST feedback. Replace missing feedback with blank strings.
eval_text['overallcomments'].fillna('', inplace=True)

# Parse Data

## Initial Data Cleaning

In [5]:
# Adding a 'document_id' column that is the row number starting from 1
eval_text = eval_text.reset_index()
eval_text['document_id'] = range(1, len(eval_text) + 1)
eval_text.set_index('document_id', inplace=True)

# Counting documents written by each supervisor
n_documents = eval_text.groupby('supervisor').size()

# Counting PSTs evaluated by each supervisor
n_psts = eval_text.groupby('supervisor')['uin_deident'].nunique()

# Joining counts back to the eval_text on supervisor
eval_text = eval_text.join(n_documents.rename('n_documents'), on='supervisor')
eval_text = eval_text.join(n_psts.rename('n_psts'), on='supervisor')

## Creating CORPUS

In [6]:
#Tokenize with SciKitLearn
engine = CountVectorizer()
model = engine.fit_transform(eval_text.overallcomments)

In [7]:
# Define a simple POS grouping function
def pos_group(tag):
    if tag.startswith('N'):
        return 'NOUN'
    elif tag.startswith('V'):
        return 'VERB'
    elif tag.startswith('J'):
        return 'ADJECTIVE'
    elif tag.startswith('R'):
        return 'ADVERB'
    else:
        return 'OTHER'

# Initialize the list to collect token data
long_format_data = []

# Iterate through each row in the DataFrame
for document_id, row in eval_text.iterrows():
    document = row['overallcomments']
    sentences = nltk.sent_tokenize(document)
    for sentence_num, sentence in enumerate(sentences):
        tokens = nltk.word_tokenize(sentence)
        tagged_tokens = nltk.pos_tag(tokens)  # Get POS tags for the tokens
        for token_num, (token, tag) in enumerate(tagged_tokens):
            long_format_data.append({
                'document_id': document_id,
                'sentence_num': sentence_num + 1,
                'token_num': token_num + 1,
                'token_str': token.lower(),  # Typically, terms are stored in lower case
                'term_str': token,           # Original token as it appears
                'pos': tag,                  # POS tag
                'pos_group': pos_group(tag)  # Grouped POS tag
            })

# Create DataFrame from long-format data
CORPUS = pd.DataFrame(long_format_data)

In [8]:
CORPUS.set_index(['document_id', 'sentence_num', 'token_num'])

token_str   term_str  pos  pos_group
document_id sentence_num token_num                                      
1           1            1          objective  Objective   NN       NOUN
                         2                  :          :    :      OTHER
                         3                 sw         SW  NNP       NOUN
                         4             recall     recall  VBP       VERB
                         5            reasons    reasons  NNS       NOUN
...                                       ...        ...  ...        ...
11385       2            36              good       good   JJ  ADJECTIVE
                         37           opening    opening   NN       NOUN
                         38               and        and   CC      OTHER
                         39           closure    closure   NN       NOUN
                         40                 .          .    .      OTHER

[1444414 rows x 4 columns]

In [9]:
CORPUS.to_csv(base_path + "output/CORPUS.csv", sep='|', index=True)

## Creating LIB

In [10]:
# Creating the new DataFrame LIB from eval_text
LIB = eval_text[['supervisor', 'uin_deident', 'order_alt', 'n_documents', 'n_psts', 'overallcomments']].copy()

# Count the number of sentences per document
sentence_counts = CORPUS.groupby('document_id')['sentence_num'].nunique().rename('sentence_count')

# Count the number of tokens per document
token_counts = CORPUS.groupby('document_id')['token_num'].size().rename('token_count')

# Combine sentence and token counts into a single DataFrame
doc_counts = pd.DataFrame({'sentence_count': sentence_counts, 'token_count': token_counts})

#Merge LIB with document count information
LIB = LIB.join(doc_counts)

# Get a count of the characters in the comments
LIB['char_count'] = eval_text['overallcomments'].str.len()

In [11]:
LIB

,supervisor,uin_deident,order_alt,n_documents,n_psts,overallcomments,sentence_count,token_count,char_count
document_id,,,,,,,,,
1,"Henry, James",1,1,172,46,Objective: SW recall reasons why the Industri...,18.0,410.0,2198
2,"Henry, James",1,2,172,46,Objective: Causes of WW1 (Many districts will...,19.0,289.0,1499
3,"Hoelscher, Nita",2,1,185,51,Very well done powerpoint on terrorism in mode...,8.0,175.0,855
4,"Opitz, Lynda",3,1,306,84,It was good to see you in action with your kid...,5.0,119.0,602
5,"Opitz, Lynda",3,2,306,84,I enjoy our work together. You are seeking wa...,7.0,147.0,772
...,...,...,...,...,...,...,...,...,...
11381,"Clark, Cindy",3251,2,103,28,Maddie wants to be a good teacher. She is an ...,2.0,23.0,116
11382,"Clark, Cindy",3251,4,103,28,Maddie is a confident student. She cares abou...,3.0,32.0,169
11383,"Clark, Cindy",3251,1,103,28,Madeline has good control of the classroom.,1.0,8.0,43


In [12]:
LIB['char_count'].fillna(0).mean()

688.8596398770312

In [13]:
LIB.to_csv(base_path + "output/LIB.csv", sep='|', index=True)

## Creating Sentence-Level Data

In [14]:
# Step 1: Aggregate terms to form sentences and count terms
grouped = CORPUS.groupby(['document_id', 'sentence_num'])
SENTENCES = pd.DataFrame({
    'sentence': grouped['term_str'].apply(' '.join),
    'term_count': grouped['term_str'].size()
}).reset_index()

# Flatten the column multi-levels generated by agg
SENTENCES.columns = ['document_id', 'sentence_num', 'sentence', 'term_count']

SENTENCES.set_index(['document_id', 'sentence_num'])

sentence  \
document_id sentence_num                                                      
1           1             Objective : SW recall reasons why the Industri...   
            2             Class began with you posing a question to the ...   
            3             ( Car ) You introduced the Industrial Revoluti...   
            4             The fact that most students arrive at school e...   
            5             You prepared and projected a power point prese...   
...                                                                     ...   
11384       6             You have been prompt , extremely reflective , ...   
            7                            Good detail on your lesson plans !   
            8                                This sets you up for success .   
11385       1             Tori appeared confident in the classroom , gre...   
            2             She referred to prior learning , modeled expec...   

                          term_count  
document_id sentence_num              
1           1                     17  
            2                     19  
            3                     26  
            4                     29  
            5                     23  
...                              ...  
11384       6                     11  
            7                      7  
            8                      7  
11385       1                     45  
            2                     40  

[91118 rows x 2 columns]

In [15]:
SENTENCES.to_csv(base_path + "output/SENTENCES.csv", sep='|', index=True)

## Creating VOCAB

In [16]:
# Calculate Term Frequency across the corpus
CORPUS['term_str'] = CORPUS['token_str'].str.lower()  # normalize to lowercase
TF = CORPUS['term_str'].value_counts().rename('n')

# Calculate Document Frequency
DF = CORPUS.groupby('term_str')['document_id'].nunique().rename('df')

# Calculate IDF using log scaling
total_documents = CORPUS['document_id'].nunique()
IDF = np.log(total_documents / DF).rename('idf')

# Calculate DFIDF
DFIDF = (DF * IDF).rename('dfidf')

# Stemming and identifying stopwords
VOCAB = pd.DataFrame(index=TF.index)
VOCAB['n'] = TF
VOCAB['df'] = DF
VOCAB['idf'] = IDF
VOCAB['dfidf'] = DFIDF
VOCAB['porter_stem'] = VOCAB.index.map(lambda x: stemmer.stem(x))
VOCAB['stop'] = VOCAB.index.isin(stop_words)

# Get max POS and POS group for each term
max_pos = CORPUS.groupby('term_str')['pos'].agg(lambda x: x.value_counts().idxmax()).rename('max_pos')
max_pos_group = CORPUS.groupby('term_str')['pos_group'].agg(lambda x: x.value_counts().idxmax()).rename('max_pos_group')

VOCAB = VOCAB.join(max_pos)
VOCAB = VOCAB.join(max_pos_group)

# Assuming handling of ngrams if applicable
# Here we assume unigram as example; modify if you have actual ngrams data
VOCAB['ngram_length'] = VOCAB.index.map(lambda x: len(x.split()))

In [17]:
VOCAB

,n,df,idf,dfidf,porter_stem,stop,max_pos,max_pos_group,ngram_length
term_str,,,,,,,,,
.,82440,10800,0.040733,439.917498,.,False,.,OTHER,1
the,66930,9659,0.152389,1471.926439,the,True,DT,OTHER,1
and,45948,10046,0.113105,1136.249721,and,True,CC,OTHER,1
to,43763,9629,0.155500,1497.308128,to,True,TO,OTHER,1
",",37686,7320,0.429669,3145.176405,",",False,",",OTHER,1
...,...,...,...,...,...,...,...,...,...
nesrsta,1,1,9.328035,9.328035,nesrsta,False,NNP,NOUN,1
'compare,1,1,9.328035,9.328035,'compar,False,POS,OTHER,1
'contrast,1,1,9.328035,9.328035,'contrast,False,POS,OTHER,1


In [18]:
VOCAB.to_csv(base_path + "output/VOCAB.csv", sep='|', index=True)

### Top 20 most significant words

In [19]:
# Sorting by DFIDF to find the top 20 significant words
top_20_significant = VOCAB.sort_values(by='dfidf', ascending=False).head(20)
top_20_significant

,n,df,idf,dfidf,porter_stem,stop,max_pos,max_pos_group,ngram_length
term_str,,,,,,,,,
she,11241,4108,1.007343,4138.164813,she,True,PRP,OTHER,1
classroom,5945,4053,1.020822,4137.391113,classroom,False,NN,NOUN,1
teacher,6408,4048,1.022056,4137.283918,teacher,False,NN,NOUN,1
were,8089,3990,1.036488,4135.587140,were,True,VBD,VERB,1
this,6439,3917,1.054953,4132.251610,thi,True,DT,OTHER,1
as,8423,4375,0.944373,4131.630633,as,True,IN,OTHER,1
are,8463,4403,0.937993,4129.983659,are,True,VBP,VERB,1
very,5814,3836,1.075849,4126.957066,veri,True,RB,ADVERB,1
her,10483,3754,1.097457,4119.854695,her,True,PRP$,OTHER,1
